In [1]:
import pandas as pd
import numpy as np

import time


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC       

In [7]:
driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.com/maps/place/Suwon+Hwaseong+Fortress,+Gyeonggi-do/data=!4m7!3m6!1s0x357b5cca0fa82ebb:0x1326e46ba3ed1641!8m2!3d37.2871202!4d127.0119379!9m1!1b1?hl=en")
driver.maximize_window()
time.sleep(10)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Sort\']'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()
time.sleep(5)

In [8]:
# scroll code
# total n review (10 reviews per 1 page)
i = 0
while i<30:
    buffer = driver.find_elements_by_class_name('qjESne') # buffer location
    time.sleep(1)
    action = ActionChains(driver)

    n =len(buffer)
    action.move_to_element(buffer[n-1]).perform() # move to last buffer
    time.sleep(1)
    i += 1

In [9]:
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

boxes = driver.find_elements_by_class_name('jftiEf.fontBodyMedium') # review class

In [10]:
# more butten click
for box in boxes:
    try:
        more_btn = box.find_element_by_class_name('w8nwRe.kyuRq')
        more_btn.click()
        time.sleep(5)
    except:
        continue
# review collection
for box in boxes:
    
    IDs = box.find_elements_by_class_name('d4r55')
        
    Dates = box.find_elements_by_class_name('rsqaWe')
        
    Stars = box.find_elements_by_class_name('kvMYJc')
        
    reviews = box.find_elements_by_class_name('wiI7pd')
    
    for ID,Date,Star,review in zip(IDs, Dates, Stars, reviews):
        name_list.append(ID.text)
        date_list.append(Date.text)
        stars_list.append(Star.get_attribute('aria-label'))
        review_list.append(review.text)

In [288]:
# review list to dataframe
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list})
pd.set_option('display.max_rows', None)
review.head(10)

,name,rating,review,date
0,Sagar Mali,5 stars,This ks Unesco World Heritage Site. Get some Y...,17 hours ago
1,KBS 4,5 stars,,2 days ago
2,onestar Jang,5 stars,,2 days ago
3,레몬쥬스,5 stars,,3 days ago
4,marina kwon,5 stars,(Translated by Google) It's cool.\n\n(Original...,3 days ago
5,Ian Jung,5 stars,"(Translated by Google) No, can historical site...",5 days ago
6,zoktras,5 stars,,5 days ago
7,PROJECT MONEY,5 stars,(Translated by Google) Great place to walk and...,6 days ago
8,Rahul V K,5 stars,,6 days ago
9,Anthony in Korea,5 stars,(Translated by Google) One of the places that ...,a week ago


In [290]:
# remove korean name
review['name2'] = review['name'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')

review = review[review['name2'].str.len() == 0]

# remove korean review
review['review2'] = review['review'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')

review = review[review['review2'].str.len() == 0]

# remove name2 & review2 columns
review.drop(['name2','review2'],axis=1, inplace=True)

# remove unreviewed data
review = review[review['review'].str.len()!=0]

# remove "(Translated by Google)" sentence
review['review'] = review['review'].str.replace('(Translated by Google) ','',regex=False)

# delete from "Original" to last word
review['review'] = review['review'].str.split('\(Original\)')
review['review'] = [item[0] for item in review['review']]

# delete side blank and "\n"
review['review'] = review['review'].str.strip()

# remove special characters
review['review'] = review['review'].str.replace(r'[^A-Za-z0-9]',' ')

# replace with lowercase
review['review'] = review['review'].str.lower()

review.head(10)

<ipython-input-290-df687ee7f502>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  review['review'] = review['review'].str.replace(r'[^A-Za-z0-9]',' ')


,name,rating,review,date
0,Sagar Mali,5 stars,this ks unesco world heritage site get some y...,17 hours ago
7,PROJECT MONEY,5 stars,great place to walk and relax,6 days ago
9,Anthony in Korea,5 stars,one of the places that i think anyone who trav...,a week ago
14,Yvonne Tan,5 stars,great walk up and gives an overview lookout of...,a week ago
19,hanimero,3 stars,i didn t go inside but i walked around slowly...,2 weeks ago
37,ritva bontinck,5 stars,nice that the room is decorated,3 weeks ago
40,jeremia msofu,5 stars,very historical place to know korean history ...,3 weeks ago
47,pagi pagi,5 stars,when i went to korea about 3 years ago i look...,a month ago
48,Nick G,5 stars,a really great time exploring the fortress wal...,a month ago
56,Shubhanginee Singh,4 stars,splendid architecture located in the city eas...,a month ago


In [287]:
review.to_csv("gmap_review.csv", encoding="utf-8")

==================================================================================